<a href="https://colab.research.google.com/github/marek-kolodziejczak98/ProjektTS2021/blob/main/projekt_ts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-statemachine

In [3]:
from statemachine import StateMachine, State
import numpy as np
import os
from IPython.display import clear_output

# deklaracja klasy
class ShrinkWrapStation(StateMachine):
    # stany
    Station_empty = State('Stacja zgrzewająca pusta', initial=True)
    Station_waiting = State('Stacja czeka na komplet butelek')
    Station_ready = State('Stacja ma zestaw butelek')
    Station_done = State('Stacja ma zgrzewkę')
    # tranzycje
    Bottle_detection = Station_empty.to(Station_waiting)
    Bottle_set_detection = Station_waiting.to(Station_ready)
    Bottle_wrapped = Station_ready.to(Station_done)
    Pack_of_bottles_out = Station_done.to(Station_empty)


class RobotSequence(StateMachine):
    # stany
    robot_in_base_position = State('Manipulator w pozycji bazowej', initial=True)
    robot_ready = State('Manipulator w pozycji gotowej do pochwycenia')
    robot_holding_beer = State('Manipulator trzymający butelkę')
    robot_in_final_position = State('Manipulator w pozycji końcowej')
    return_to_wrapping_process = State('Powrót do procesu zgrzewania')

    # tranzycje
    bottle_detection = robot_in_base_position.to(robot_ready)
    bottle_grabbed = robot_ready.to(robot_holding_beer)
    move_bottle = robot_holding_beer.to(robot_in_final_position)
    return_to_the_base_pos = robot_in_final_position.to(return_to_wrapping_process)

# stworzenie modelu
Shrink_Wrap_Station = ShrinkWrapStation()
# stworzenie wektorow zawierajacych nazwy stanow i tranzycji
# stany tworzymy jako liste stringow, poniewaz kazdy stan ma swoja nazwe
States = ["Station_empty", "Station_waiting", "Station_ready", "Station_done"]
# tranzycje tworzymy jako liste stringow jesli jest jedna tranzycja, i list stringow jesli jest ich wiecej niz 1
Transitions = ["Bottle_detection", ("Bottle_set_detection", "Full_set_not_detected"), "Bottle_wrapped", "Pack_of_bottles_out"]

robot_sequence_model = RobotSequence()
rStates = ["Robot in base position", "Robot ready to grab a bottle", "Robot holding a bottle", "Robot in the final position", "Return to the wrapping process"]
rTransitions = ["Bottle detection", "Bottle grabbed", "move bottle", "Return to the base position"]

In [ ]:
# nieskonczona petla
while True:
    # pobranie identyfikatora aktualnego stanu
    current_state = Shrink_Wrap_Station.current_state.identifier
    # stworzenie pustej zmiennej do ktorej wczytane zostana dostepne dla aktualnego stanu tranzycje
    current_transitions = []
    # wyswietlenie nazwy aktualnego stanu
    print("Current state: "+str(Shrink_Wrap_Station.current_state.name))
    # sprawdzenie dostepnych tranzycji dla obecnego stanu
    for i in range(len(States)):
        # robimy petle przechodzaca po wszystkich stanach
        state = States[i]
        # sprawdzamy czy sprawdzany stan jest rowny obecnemu stanowi
        if state == current_state:
            # ustawiamy aktualnie dostepne tranzycje z wektora tranzycji dzieki odpowiedniemu indeksowaniu
            current_transitions = Transitions[i]
            break
    # wyswietlamy aktualnie dostepne tranzycje
    print("\nAvailable transitions: ")
    print("0. Exit")
    # jesli jest tylko jedna tranzycja zmienna current_transitions jest stringiem, w przypadku wiekszej niz jednej tranzycji zmienna ta jest lista
    # zamiast sprawdzac dlugosc tej zmiennej trzeba sprawdzic, czy jest ona stringiem, poniewaz dlugosc nie zwroci nam 1, tylko dlugosc stringa
    if isinstance(current_transitions, str):
        print("1. "+current_transitions)
    else:
        # jesli current_transitions nie jest stringiem, to znaczy ze jest lista, wiec mozemy iterowac po wszystkich jej elementach
        for i in range(len(current_transitions)):
             print(str(i+1)+". "+current_transitions[i])
    # prosimy uzytkownika o podanie liczby
    decision = input("\nPick transition: ")
    if decision.isdigit() and int(decision) == 0:
        break
    # sprawdzamy ponownie czy dostepna jest jedna czy wiecej tranzycji
    if isinstance(current_transitions, str) and decision.isdigit() and int(decision) == 1:
        # deklarujemy funkcje tran() ktora pozwala wykorzystac nazwe zapisana w stringu(current_transitions) jako czesc funkcji
        # w rezultacie tran() jest rownoznaczne Shrink_Wrap_Station.current_transitions, gdzie current_transitions to string z nazwa tranzycji
        tran = getattr(Shrink_Wrap_Station, current_transitions)
        tran()
        clear_output()
    elif  decision.isdigit() and not isinstance(current_transitions, str) and int(decision)<=len(current_transitions):
        tran = getattr(Shrink_Wrap_Station, current_transitions[int(decision)-1])
        tran()
        clear_output()
    else:
        clear_output()
        print("What you typed was wrong!\n")
clear_output()
print("Program zakończony poprawnie")

Program zakończony poprawnie
